# GPs on Non-Euclidean Input Spaces
GPs on non-Euclidean input spaces have become more and more relevant in recent years, especially for Bayesian Optimization in chemistry. gpCAM can be used for that purpose as long as a correct kernel is defined. Of course, if mean and noise functions are also provided, they have to operate on these non-Euclidean spaces as well. 

In this example, we run a small GP on words.

In [1]:
#install the newest version of fvgp
#!pip install gpcam==8.0.0

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from gpcam import GPOptimizer
from dask.distributed import Client
%load_ext autoreload
%autoreload 2

In [3]:
#making the x_data a set will allow us to put any objects or structures into it.
x_data = {('hello'),('world'),('this'),('is'),('fvgp')}
y_data = np.array([2.,1.9,1.8,3.0,5.])


def string_distance(string1, string2):
    difference = abs(len(string1) - len(string2))
    common_length = min(len(string1),len(string2))
    string1 = string1[0:common_length]
    string2 = string2[0:common_length]
    
    for i in range(len(string1)):
        if string1[i] != string2[i]:
            difference += 1.

    return difference


def kernel(x1,x2,hps,obj):
    d = np.zeros((len(x1),len(x2)))
    count1 = 0
    for string1 in x1:
        count2 = 0
        for string2 in x2:
            d[count1,count2] = string_distance(string1,string2)
            count2 += 1
        count1 += 1
    return hps[0] * obj.matern_kernel_diff1(d,hps[1])
    

bounds = np.array([[0.001,100.],[0.001,100]])
my_gp = GPOptimizer(x_data,y_data, init_hyperparameters=np.ones((2)),
                    gp_kernel_function=kernel,hyperparameter_bounds=bounds, info = True)
my_gp.train()

print("hyperparameters: ", my_gp.hyperparameters)
print("prediction : ",my_gp.posterior_mean({'full'})["f(x)"])
print("uncertainty: ",np.sqrt(my_gp.posterior_covariance({'full'})["v(x)"]))

differential_evolution step 1: f(x)= 10.1306
differential_evolution step 2: f(x)= 8.85223
differential_evolution step 3: f(x)= 8.85223
differential_evolution step 4: f(x)= 8.85223
differential_evolution step 5: f(x)= 8.72559
differential_evolution step 6: f(x)= 8.72559
differential_evolution step 7: f(x)= 8.10705
differential_evolution step 8: f(x)= 8.10705
differential_evolution step 9: f(x)= 8.10705
differential_evolution step 10: f(x)= 8.10705
differential_evolution step 11: f(x)= 8.10705
differential_evolution step 12: f(x)= 8.10705
differential_evolution step 13: f(x)= 8.066
differential_evolution step 14: f(x)= 8.04529
differential_evolution step 15: f(x)= 8.04529
differential_evolution step 16: f(x)= 8.04529
differential_evolution step 17: f(x)= 8.04529
differential_evolution step 18: f(x)= 8.04529
differential_evolution step 19: f(x)= 8.04529
differential_evolution step 20: f(x)= 8.04497
differential_evolution step 21: f(x)= 8.04489
differential_evolution step 22: f(x)= 8.04489

In [4]:
##which one should I measure next?
my_gp.ask(candidate_set={('hello'),('world'),("it"),("is"),("me")}, n = 4)

{'x': array([['me'],
        ['it'],
        ['is'],
        ['world']], dtype='<U5'),
 'f(x)': array([1.43588541, 1.43588541, 0.02688694, 0.02688694]),
 'opt_obj': None}